In [205]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from datasets import load_dataset
from timm import create_model
from transformers import ViTImageProcessor
from transformers import ViTConfig, ViTModel
import evaluate 
import numpy as np


In [206]:
from transformers import ViTImageProcessor

model_name_or_path = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTImageProcessor.from_pretrained(model_name_or_path, return_tensors='pt')


In [207]:
# Initializing a ViT vit-base-patch16-224 style configuration
configuration = ViTConfig()

# Initializing a model (with random weights) from the vit-base-patch16-224 style configuration
model = ViTModel(configuration)



In [208]:
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x for x in example_batch['image']], return_tensors='pt')

    # Don't forget to include the labels!
    inputs['labels'] = example_batch['label']
    return inputs


In [209]:
ds = load_dataset("C:/Tesis/DatasetBinario", num_proc=3)

Resolving data files:   0%|          | 0/20959 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/668 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/768 [00:00<?, ?it/s]

In [210]:
dataset_train = ds['train']
dataset_test = ds['test']
num_classes = len(set(dataset_train['label']))
labels = ds['train'].features['label']
print(num_classes, labels)

2 ClassLabel(names=['Melanoma', 'No Melanoma'], id=None)


In [211]:
example = feature_extractor(dataset_train[0]['image'], return_tensors = 'pt')

In [212]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [213]:
def preprocess(batch):
    inputs = feature_extractor(batch['image'], return_tensors='pt').to(device)
    inputs['label'] = batch['label']
    return inputs

In [214]:
prepared_train = dataset_train.with_transform(preprocess)
prepared_test = dataset_test.with_transform(preprocess)

In [215]:
print(prepared_train.features)

{'image': Image(mode=None, decode=True, id=None), 'label': ClassLabel(names=['Melanoma', 'No Melanoma'], id=None)}


In [216]:
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }


In [217]:
from transformers import ViTForImageClassification

labels = ds['train'].features['label'].names

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
).to(device)


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [218]:
metric = evaluate.load("accuracy")

In [219]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [220]:
from transformers import TrainingArguments, Trainer


training_args = TrainingArguments(
    output_dir='./results', 
    per_device_eval_batch_size=16,
    evaluation_strategy="steps",
    num_train_epochs=4,
    save_steps=1000,
    eval_steps=500,
    logging_steps=10,
    learning_rate=2e-5,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=False,
    load_best_model_at_end=True,
    dataloader_pin_memory=False
    
    )

In [221]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= prepared_train,
    eval_dataset= prepared_test,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
    tokenizer=feature_extractor
)

C:\Users\abadd\AppData\Local\Temp\ipykernel_24276\1455688735.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [222]:
trainer.train()

  0%|          | 0/10480 [00:00<?, ?it/s]

{'loss': 0.6905, 'grad_norm': 1.552635669708252, 'learning_rate': 1.9980916030534353e-05, 'epoch': 0.0}
{'loss': 0.6801, 'grad_norm': 2.5676097869873047, 'learning_rate': 1.9961832061068704e-05, 'epoch': 0.01}
{'loss': 0.6664, 'grad_norm': 2.0010132789611816, 'learning_rate': 1.9942748091603055e-05, 'epoch': 0.01}
{'loss': 0.6459, 'grad_norm': 2.212742567062378, 'learning_rate': 1.9923664122137406e-05, 'epoch': 0.02}
{'loss': 0.6522, 'grad_norm': 2.185227155685425, 'learning_rate': 1.9904580152671757e-05, 'epoch': 0.02}
{'loss': 0.6228, 'grad_norm': 1.8474783897399902, 'learning_rate': 1.988549618320611e-05, 'epoch': 0.02}
{'loss': 0.6154, 'grad_norm': 2.6391870975494385, 'learning_rate': 1.986641221374046e-05, 'epoch': 0.03}
{'loss': 0.5935, 'grad_norm': 2.31415057182312, 'learning_rate': 1.984732824427481e-05, 'epoch': 0.03}
{'loss': 0.6021, 'grad_norm': 2.0074756145477295, 'learning_rate': 1.9828244274809162e-05, 'epoch': 0.03}
{'loss': 0.5949, 'grad_norm': 3.152873992919922, 'learn

  0%|          | 0/48 [00:00<?, ?it/s]

{'eval_loss': 0.47657182812690735, 'eval_accuracy': 0.7890625, 'eval_runtime': 38.4816, 'eval_samples_per_second': 19.958, 'eval_steps_per_second': 1.247, 'epoch': 0.19}
{'loss': 0.5834, 'grad_norm': 1.3982537984848022, 'learning_rate': 1.902671755725191e-05, 'epoch': 0.19}
{'loss': 0.4017, 'grad_norm': 11.956353187561035, 'learning_rate': 1.900763358778626e-05, 'epoch': 0.2}
{'loss': 0.3618, 'grad_norm': 4.305224418640137, 'learning_rate': 1.8988549618320614e-05, 'epoch': 0.2}
{'loss': 0.5166, 'grad_norm': 4.460570812225342, 'learning_rate': 1.8969465648854962e-05, 'epoch': 0.21}
{'loss': 0.5058, 'grad_norm': 8.367267608642578, 'learning_rate': 1.8950381679389313e-05, 'epoch': 0.21}
{'loss': 0.5071, 'grad_norm': 15.515195846557617, 'learning_rate': 1.8931297709923668e-05, 'epoch': 0.21}
{'loss': 0.5324, 'grad_norm': 5.047912120819092, 'learning_rate': 1.8912213740458016e-05, 'epoch': 0.22}
{'loss': 0.5067, 'grad_norm': 4.497890472412109, 'learning_rate': 1.889312977099237e-05, 'epoch'

  0%|          | 0/48 [00:00<?, ?it/s]

...

  0%|          | 0/48 [00:00<?, ?it/s]

{'eval_loss': 0.37487515807151794, 'eval_accuracy': 0.9114583333333334, 'eval_runtime': 76.893, 'eval_samples_per_second': 9.988, 'eval_steps_per_second': 0.624, 'epoch': 3.63}
{'loss': 0.0041, 'grad_norm': 0.04281257838010788, 'learning_rate': 1.8511450381679392e-06, 'epoch': 3.63}
{'loss': 0.027, 'grad_norm': 0.8221969604492188, 'learning_rate': 1.8320610687022903e-06, 'epoch': 3.63}
{'loss': 0.0273, 'grad_norm': 0.026127588003873825, 'learning_rate': 1.8129770992366414e-06, 'epoch': 3.64}
{'loss': 0.2178, 'grad_norm': 8.104713439941406, 'learning_rate': 1.7938931297709925e-06, 'epoch': 3.64}
{'loss': 0.0035, 'grad_norm': 0.024140330031514168, 'learning_rate': 1.7748091603053436e-06, 'epoch': 3.65}
{'loss': 0.0893, 'grad_norm': 1.6791025400161743, 'learning_rate': 1.7557251908396948e-06, 'epoch': 3.65}
{'loss': 0.1043, 'grad_norm': 0.22810927033424377, 'learning_rate': 1.736641221374046e-06, 'epoch': 3.65}
{'loss': 0.1035, 'grad_norm': 0.0848689004778862, 'learning_rate': 1.717557251

  0%|          | 0/48 [00:00<?, ?it/s]

{'eval_loss': 0.4121273458003998, 'eval_accuracy': 0.9010416666666666, 'eval_runtime': 70.8672, 'eval_samples_per_second': 10.837, 'eval_steps_per_second': 0.677, 'epoch': 3.82}
{'loss': 0.0706, 'grad_norm': 0.02534552291035652, 'learning_rate': 8.969465648854963e-07, 'epoch': 3.82}
{'loss': 0.1329, 'grad_norm': 0.040772199630737305, 'learning_rate': 8.778625954198474e-07, 'epoch': 3.82}
{'loss': 0.0526, 'grad_norm': 0.0337890163064003, 'learning_rate': 8.587786259541986e-07, 'epoch': 3.83}
{'loss': 0.0036, 'grad_norm': 0.038144223392009735, 'learning_rate': 8.396946564885497e-07, 'epoch': 3.83}
{'loss': 0.1351, 'grad_norm': 0.027762508019804955, 'learning_rate': 8.206106870229009e-07, 'epoch': 3.84}
{'loss': 0.003, 'grad_norm': 0.031778834760189056, 'learning_rate': 8.01526717557252e-07, 'epoch': 3.84}
{'loss': 0.0065, 'grad_norm': 0.11197499185800552, 'learning_rate': 7.824427480916032e-07, 'epoch': 3.84}
{'loss': 0.1622, 'grad_norm': 0.0315922275185585, 'learning_rate': 7.6335877862

TrainOutput(global_step=10480, training_loss=0.23013947155212855, metrics={'train_runtime': 22792.3306, 'train_samples_per_second': 3.678, 'train_steps_per_second': 0.46, 'total_flos': 6.496618441328935e+18, 'train_loss': 0.23013947155212855, 'epoch': 4.0})

In [ ]:
trainer.save_model()
trainer.save_state()


In [226]:
prepared_val = ds['validation'].with_transform(preprocess)

  0%|          | 0/42 [00:00<?, ?it/s]

{'eval_loss': 0.3163492679595947,
 'eval_accuracy': 0.907185628742515,
 'eval_runtime': 70.2505,
 'eval_samples_per_second': 9.509,
 'eval_steps_per_second': 0.598,
 'epoch': 4.0}

: 